In [1]:
import cv2
import os
import numpy as np

#os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [2]:
for sub_dir in sub_dirs:
    for category in categories:
        category_path = os.path.join(base_dir, sub_dir, category)
        images[sub_dir][category] = []
        
        if os.path.exists(category_path):
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                img = cv2.imread(img_path)

                img = cv2.resize(img, resize_dim)
                # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                images[sub_dir][category].append(img)
        else:
            print(f"Category path does not exist: {category_path}")


NameError: name 'sub_dirs' is not defined

In [ ]:
test_tomato_leaf_images = images['test']['Tomato leaf']

print(f"Loaded {len(test_tomato_leaf_images)} Tomato leaf images from the Testing set.")

Loaded 8 Tomato leaf images from the Testing set.


In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np

2024-03-24 21:42:42.731140: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 21:42:42.731347: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/raag/sdp/env/lib/python3.8/site-packages/cv2/../../lib64:
2024-03-24 21:42:42.731415: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/raag/sdp/env/lib/python3.8/site-packages/cv2/../../lib64:
2024-03-24 21:42:42.731451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: li

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)  # Dropout layer to reduce overfitting

# Add global average pooling layer to further reduce dimensionality
feature_extractor = base_model.output
feature_extractor = GlobalAveragePooling2D()(feature_extractor)

# This model outputs the extracted features
feature_extraction_model = Model(inputs=base_model.input, outputs=feature_extractor)

feature_extraction_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
import cirq
import sympy
import tensorflow_quantum as tfq

featSize = 6

def create_quantum_model():
    # Define a small set of qubits
    qubits = [cirq.GridQubit(0, i) for i in range(featSize)]  # Adjust based on feature size

    # Build the quantum circuit
    circuit = cirq.Circuit()
    for qubit in qubits:
        circuit.append(cirq.H(qubit))  # Hadamard gate for superposition
        theta = sympy.Symbol(f"theta_{qubit}")
        circuit.append(cirq.rz(theta)(qubit))  # Rotation Z gate with parameter theta
        # Entangle qubits as needed
        if qubit != qubits[-1]:
            circuit.append(cirq.CNOT(qubit, qubits[qubits.index(qubit) + 1]))

    # Observable for the readout
    observable = cirq.Z(qubits[-1])  # Example observable

    return circuit, observable

# Quantum circuit and observable
qcircuit, observable = create_quantum_model()

# Quantum Keras layer
quantum_layer = tfq.layers.PQC(qcircuit, operators=observable)


In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Setup a learning rate reduction strategy
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

# Classical feature extraction
inputs = Input(shape=(224, 224, 3))
extracted_features = feature_extraction_model(inputs)

# Placeholder for the processed features to be fed into the quantum model
# This step would involve encoding the features into quantum states
quantum_input = tf.keras.layers.Lambda(lambda x: tfq.convert_to_tensor([cirq.Circuit()]))(extracted_features)

# Quantum model processing
quantum_output = quantum_layer(quantum_input)

initial_learning_rate = 1e-4
optimizer = Adam(learning_rate=initial_learning_rate)

# Final model
model = Model(inputs=[inputs], outputs=[quantum_output])

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 model (Functional)          (None, 1280)              2257984   
                                                                 
 lambda (Lambda)             (1,)                      0         
                                                                 
 pqc (PQC)                   (1, 1)                    6         
                                                                 
Total params: 2,257,990
Trainable params: 2,223,878
Non-trainable params: 34,112
_________________________________________________________________


In [ ]:
# Assuming 'model' is your hybrid model prepared earlier
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Number of epochs to train for
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
58/58 [==============================] - 33s 536ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 2/10
58/58 [==============================] - 29s 497ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 3/10
58/58 [==============================] - 29s 497ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 4/10
58/58 [==============================] - 29s 489ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 5/10
58/58 [==============================] - 28s 485ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 6/10
58/58 [==============================] - 30s 511ms/step - loss: 1.1921e-07 - accuracy: 0.9643 - val_loss: 1.1921e-07 - val_accuracy: 0.9643
Epoch 7/10
58/58 [==============================] - 29s 495ms/step - loss: 1.1921e-07 - accura

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


8/8 [==============================] - 2s 238ms/step - loss: 1.1921e-07 - accuracy: 0.9630
Test Loss: 1.1920928955078125e-07
Test Accuracy: 0.9629630446434021
